In [16]:
import os

In [18]:
pwd

'd:\\Github\\Genia\\Final_pjt2\\Lecture_Text'

In [17]:
# Change the working directory
os.chdir('./Lecture_Text')

## Hugging Face

- NCIC

- Lecture

In [21]:
# eenzeenee/t5-base-korean-summarization

import nltk
nltk.download('punkt')
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained('eenzeenee/t5-base-korean-summarization')
tokenizer = AutoTokenizer.from_pretrained('eenzeenee/t5-base-korean-summarization')

# Specify the file path
file_path = './txt/01강유리수의소수표현(1)_EBS중학뉴런수학2(상).txt'

# Read the text from the file
with open(file_path, 'r') as file:
    text = file.read()
print(f'자막 원본 길이 : {len(text)}')

# inputs = [prefix + sample]
inputs = [text]

inputs = tokenizer(inputs, max_length=512, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=3, do_sample=True, min_length=1000, max_length=10000) 
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
result = nltk.sent_tokenize(decoded_output.strip())[0]

print('RESULT >>', result)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hslio\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


자막 원본 길이 : 17571
RESULT >> 뉴런 수학은 수의 관한 내용과 식에 관한 내용 그리고 함수로 이루어져 있고 이 내용 중에서 처음 공부하는 단원은 유리수와 소수라는 단원이 될 것이며 첫 번째 공부할 내용은 유리수와 소수에 대해 알아볼 것이고 두 번째는 순환 소수에 대해 알아볼 것이다.


### Translate For Summarize
- Kor -> Eng -> Summarize -> Kor
- Translate : https://huggingface.co/facebook/mbart-large-50-many-to-many-mmt 
- Summarize : https://huggingface.co/facebook/bart-large-cnn 


In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from transformers import pipeline

In [73]:
# Specify the file path
file_path = './txt/01강유리수의소수표현(1)_EBS중학뉴런수학2(상).txt'

# Read the text from the file
with open(file_path, 'r') as file:
    text = file.read()

print(f'자막 원본 길이 : {len(text)}')

# Define a function to split the text into chunks
def chunk_text(text, chunk_size):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

def google_trans_to_ko(messages):
    google = Translator()
    result = google.translate(messages, dest="ko")
    return result.text

def google_trans_to_en(messages):
    google = Translator()
    result = google.translate(messages, dest="en")
    return result.text

# Specify the chunk size
chunk_size = 500  # Adjust this value based on your needs

# Split the text into chunks
chunks = chunk_text(text, chunk_size)

# Initialize the final result
final_result = ''

자막 원본 길이 : 17571


facebook/mbart-large-50-many-to-many-mmt -> facebook/bart-large-cnn -> facebook/mbart-large-50-many-to-many-mmt

In [55]:
# Process each chunk
for chunk in chunks:
    #####
    # Translate Korean to English
    input = chunk

    model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
    tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
    
    tokenizer.src_lang = "ko_KR"
    encoded_hi = tokenizer(input, return_tensors="pt")
    generated_tokens = model.generate(
        **encoded_hi,
        forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
    )
    to_eng = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    
    #####
    # Summarize English
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    ARTICLE = to_eng[0]
    eng_sum = summarizer(ARTICLE, do_sample=False)
    
    #####
    # Translate English to Korean
    input = eng_sum[0]['summary_text']
    
    model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
    tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
    
    tokenizer.src_lang = "en_XX"
    encoded_hi = tokenizer(input, return_tensors="pt")
    generated_tokens = model.generate(
        **encoded_hi,
        forced_bos_token_id=tokenizer.lang_code_to_id["ko_KR"]
    )
    result = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

    # Append the result to the final result
    final_result += result[0]

print(final_result)
print(f'최종 요약 길이 : {len(final_result)}')

Your max_length is set to 142, but your input_length is only 136. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=68)
Your max_length is set to 142, but your input_length is only 27. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)


샌드맨은 수학 수학 신경학의 2학년입니다. 우리가 2학년에서 할 일 중 가장 중요한 것은 숫자, 음식, 그리고 기능을 배울 것입니다. 우리 강의의 제목은 수학: 뉴런이란 무엇일까요? 반지란 무엇일까요? 우리는 달리고 배우고 있습니다.유리 숫자는 분자의 형태로 표현할 수 있는 숫자입니다. 우리가 3학년 때부터 좀 더 정교하게 얘기할 것입니다. 우리는 유리 숫자와 소수에 대해 배웠습니다. 그리고 두번째로, 새로운 것이 생겨나고 있습니다.b는 0이 필요하지 않습니다. 그래서 우리는 이 숫자를 유리 숫자로 표현할 수 있는 숫자라고 부릅니다. 그래서 지금까지 우리가 알고 있는 거의 모든 숫자들은 그래프로 표현할 수 있습니다. 하지만 그 중 하나를 표현하기가 어려운 경우, 첫번째 반에 배운 π가 그 숫자일까요?우리는 숫자를 '정한'이라고 부릅니다. 그리고 영원히 지속되는 숫자가 있습니다. 우리는 이 무한한 숫자를 '영원히'라고 부릅니다. 그리고 여러분의 조상들이 여러분에게 물었을 때, "교수님, 왜 이런가요? 0이 아니라 무한한 숫자입니다. 왜 이렇게 부릅니다?""이것이 영원히 지속된다면, 이것은 무한하다." 여러분은 그렇게 할 수 있습니다. 그래서 우리는 "아니요, 무한하다"라고 말하지 않는 이유입니다. 무수보다 아래에 무한한 숫자가 있다는 의미입니다. 그래서 "이것이 무한하지 않나요?"라고 생각하기 쉽습니다. 왜냐하면 그 중간에 0이 있기 때문이죠.그 뒤에는 아무것도 없죠. 그리고 여러분은 "엄마, 0.3333 다음에는 세 개가 더 있을거야." 라고 생각하죠. 만약 더 많은 것을 원한다면, 작은 점을 추가해야만 합니다. 하지만 아니죠. 그 작은 점은 소수 아래 네번째 자리에 끝납니다. 그래서 무한의 숫자입니다."주기적인 숫자란 무엇일까요? 그게 제가 2학년 때 만들어낸 첫 번째 용어입니다."라고 그는 말합니다. 지구상의 공기는, 우리 나라의 꼭대기에 매일 우리 나라 위에 있는 것이 아닙니다. 낮은 압력, 높은 압력, 북태평양, 시베리아의 낮은 압력 등등이 있습니다. 

----

google_trans -> facebook/bart-large-cnn -> google_trans

In [88]:
# Define a function to split the text into chunks
def chunk_text(text, chunk_size):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

def google_trans_to_ko(messages):
    google = Translator()
    result = google.translate(messages, dest="ko")
    return result.text

def google_trans_to_en(messages):
    google = Translator()
    result = google.translate(messages, dest="en")
    return result.text

자막 원본 길이 : 17571


In [93]:
# Specify the file path
file_path = './중학 뉴런 수학 2학년 (상) 손석민_all.txt'

# Read the text from the file
with open(file_path, 'r') as file:
    text = file.read()

# Specify the chunk size
chunk_size = 400  # Adjust this value based on your needs

# Split the text into chunks
chunks = chunk_text(text, chunk_size)

# Initialize the final result
final_result = ''

print(f'자막 원본 길이 : {len(text)}')

자막 원본 길이 : 805141


In [94]:
# Process each chunk
for chunk in chunks:
    #####
    # Translate Korean to English
    input = chunk
    print(f'원본 데이터 :{input}')
    to_eng = google_trans_to_en(input)
    print(f'영어 변환 :{to_eng}')

    #####
    # Summarize English
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    ARTICLE = to_eng
    eng_sum = summarizer(ARTICLE, do_sample=False)
    print(f'영어 요약 :{to_eng}')
    
    #####
    # Translate English to Korean
    input = eng_sum[0]['summary_text']

    result = google_trans_to_ko(input)
    print(f'한글 결과 :{result}')

    # Append the result to the final result
    final_result += result
    print('<--------------------------------------->')

print(final_result)
print(f'최종 요약 길이 : {len(final_result)}')

원본 데이터 :반갑습니다, EBS 친구 여러분들. 중학 뉴런 수학 2학년 (상)의 손석민입니다. 여러분들과 함께 새로운 강좌로 만나게 되었습니다. 일단 반갑다는 인사를 드리고 싶고요.
우리 교재 제목, 뉴런 수학이죠? ‘런’이 뭐예요? 달리기도 있고 배우는 것도 있어요. ‘뉴’는 뭐예요? ‘새로운’입니다.
우리가 수학을 새롭게 공부해 볼 거고, 새롭게 달린다는 마음으로, 2학년이 되었으니까 그렇게 공부를 했으면 하는 선생님의 바람입니다.
우리가 2학년 (상)에서 주로 공부하게 될 내용은 바로 수의 관한 내용, 식에 관한 내용, 함수 이렇게 이루어져 있는데요. 이 내용 중에서 우리 그럼 처음 공부하는 단원은 바로 ‘유리수와 소수’라는 단원이 되겠고요.
그래서 우리 첫 번째 공부할 주제를 잡아 보면 바로 ‘유리수의 소수’ 편
영어 변환 :Nice to meet you, EBS friends. This is Son Seok-min, a second-year middle school student in Neuron Mathematics. We have a new course with you all. First of all, I would like to say hello to you.
The title of our textbook is Neuron Mathematics, right? What is ‘Run’? There is running and there is learning. What is ‘new’? It is ‘new’.
The teacher's hope is that we will study mathematics in a new way, and that we will study like that since we are in second grade with the mindset of having a fresh start.
The content we will mainly study in the second year (upper) consists of numbers

KeyboardInterrupt: 

In [90]:
final_result

"손석민은 뉴런수학과 중학교 2학년이다. 2학년 때 주로 공부할 내용은 숫자, 표현, 함수로 구성됩니다. 이 내용 중 첫 번째로 공부할 단원은 '유리수와 소수'입니다.유리수는 분수로 표현할 수 있는 숫자입니다. 원형 소수는 원형 모양의 숫자입니다. 유리수와 소수를 먼저 공부하겠습니다. 그런 다음 유리수를 다루기 위해 순환소수를 사용하는 방법을 배우겠습니다. 안돼요. 아니 알겠습니다.예를 들어 5/3, 3, 5는 정수이고, 마이너스 7/5, 5, -7은 이렇게 생각할 수 있습니다. 3이 되면 상황이 조금 더 복잡해지기 때문에 지금 우리가 알고 있는 분수는 분수이고, 이렇게 이야기하면 b와 a는 모두 정수형입니다.유한십진수는 극한이 있는 십진수입니다. 우리는 이러한 소수를 유한소수라고 부르는데, 끝없이 이어지는 소수도 있습니다. 첫 번째는 유한소수이고, 두 번째는 무한소수이다. 부모의 사랑에는 한계가 없습니다. 무엇? 끝이 없단 말이에요.즉, 갈 수 있는 곳은 끝이 없습니다. 계속해서 이어지는 이러한 소수를 무한소수라고 합니다. 예를 들어 0.333… 이것은 무엇을 의미하나요? 이는 숫자 3이 끝없이 계속된다는 뜻이다. 그리고 나는 끝없이 나아갈 것이다. 6.252525… 그리고 끝없이 이어지는데,교사: 0.2는 정말 유한십진수인가요? 나는 그렇게 생각하지 않습니다. 무한대로 가면 무한한 십진수이다. 그럼 0이 아닌 숫자만 있어야 한다고 생각하시나요? 그렇다면 0.205205… 이것은 무한소수가 아닌가? 그냥 0이 이렇게 끝없이 계속되는 뿐입니다.소수점 이하 네 번째 자리에서 끝나므로 유한소수이다. 무한소수는 어차피 끝없이 이어지기 때문에 보통 딩딩딩딩처럼 뒤에 붙는다고 생각하면 된다. 아주 쉽게 확인하실 수 있습니다. 뒤에는 아무것도 없습니다. 0.454545.우리는 유리수가 무엇인지 배웠고 소수는 유한과 무한으로 나뉩니다. 우리는 실제 소수를 살펴보고 유한과 무한을 구별했습니다. 이제 우리가 공부할 다음 것은 Decimals를 반복하는 것입니다. 2학년 입학 

In [91]:
import textwrap

with open('final_result.txt', 'w') as f:
    f.write('\n'.join(textwrap.wrap(final_result, width=80)))